# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

[2025-11-09 05:32:08] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-09 05:32:08] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-09 05:32:08] INFO utils.py:164: NumExpr defaulting to 16 threads.


In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-09 05:32:15] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:32:15] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:32:15] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:32:19] WARNING server_args.py:1186: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-09 05:32:19] WARNING server_args.py:1559: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-09 05:32:19] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 05:32:29] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:32:29] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:32:29] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:32:31] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


...

Capturing batches (bs=4 avail_mem=18.54 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=3 avail_mem=18.38 GB):  25%|██▌       | 1/4 [00:05<00:15,  5.19s/it]

Capturing batches (bs=1 avail_mem=18.35 GB):  75%|███████▌  | 3/4 [00:06<00:01,  1.49s/it]

Capturing batches (bs=1 avail_mem=18.35 GB): 100%|██████████| 4/4 [00:10<00:00,  2.59s/it]


Capturing batches (bs=1 avail_mem=53.58 GB): 100%|██████████| 4/4 [00:00<00:00, 24.41it/s]


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-09 05:33:33] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:33:33] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:33:33] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:33:35] WARNING server_args.py:1186: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-09 05:33:35] WARNING server_args.py:1559: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-09 05:33:35] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 05:33:44] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:33:44] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:33:44] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:33:46] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


...

Capturing batches (bs=3 avail_mem=32.45 GB):  25%|██▌       | 1/4 [00:07<00:23,  7.87s/it]

Capturing batches (bs=2 avail_mem=53.55 GB):  50%|█████     | 2/4 [00:09<00:07,  3.99s/it]

Capturing batches (bs=1 avail_mem=53.51 GB):  75%|███████▌  | 3/4 [00:09<00:02,  2.32s/it]

Capturing batches (bs=1 avail_mem=53.51 GB): 100%|██████████| 4/4 [00:15<00:00,  3.83s/it]


Capturing batches (bs=1 avail_mem=52.86 GB): 100%|██████████| 4/4 [00:00<00:00, 80.14it/s]


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16  --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-09 05:35:32] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:35:32] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:35:32] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:35:35] WARNING model_config.py:880: Casting torch.bfloat16 to torch.float16.
[2025-11-09 05:35:35] WARNING server_args.py:1186: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-09 05:35:35] WARNING server_args.py:1559: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-09 05:35:35] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 05:35:40] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:35:40] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:35:40] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:35:42] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 05:35:43] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=4 avail_mem=58.07 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=3 avail_mem=57.96 GB):  25%|██▌       | 1/4 [00:03<00:11,  3.71s/it]

Capturing batches (bs=1 avail_mem=57.89 GB):  75%|███████▌  | 3/4 [00:04<00:01,  1.07s/it]

Capturing batches (bs=1 avail_mem=57.89 GB): 100%|██████████| 4/4 [00:06<00:00,  1.69s/it]


Capturing batches (bs=1 avail_mem=57.75 GB): 100%|██████████| 4/4 [00:00<00:00, 89.20it/s]


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-09 05:36:47] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:36:47] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:36:47] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:36:49] WARNING model_config.py:880: Casting torch.bfloat16 to torch.float16.
[2025-11-09 05:36:50] WARNING server_args.py:1186: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-09 05:36:50] WARNING server_args.py:1559: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-09 05:36:50] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 05:36:56] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:36:56] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:36:56] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:36:58] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 05:36:58] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=4 avail_mem=56.98 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=3 avail_mem=57.25 GB):  25%|██▌       | 1/4 [00:02<00:07,  2.51s/it]

Capturing batches (bs=1 avail_mem=57.17 GB):  75%|███████▌  | 3/4 [00:02<00:00,  1.38it/s]

Capturing batches (bs=1 avail_mem=57.17 GB): 100%|██████████| 4/4 [00:04<00:00,  1.20s/it]


Capturing batches (bs=1 avail_mem=57.02 GB): 100%|██████████| 4/4 [00:00<00:00, 103.26it/s]


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../basic_usage/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5 --log-level warning
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-11-09 05:37:59] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:37:59] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:37:59] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:38:02] WARNING server_args.py:1186: Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-11-09 05:38:02] WARNING server_args.py:1559: Overlap scheduler is disabled because of using eagle3 or standalone speculative decoding.
[2025-11-09 05:38:02] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 05:38:09] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:38:09] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:38:09] INFO utils.py:164: NumExpr defaulting to 16 threads.
[2025-11-09 05:38:09] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2025-11-09 05:38:09] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
[2025-11-09 05:38:09] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 05:38:11] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 05:38:11] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


...

Capturing batches (bs=1 avail_mem=59.71 GB): 100%|██████████| 4/4 [00:01<00:00,  3.95it/s]


[2025-11-09 05:38:42] SPECULATIVE_MOE_RUNNER_BACKEND is not initialized, using auto backend
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.09it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  6.70it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  6.39it/s]



Capturing batches (bs=1 avail_mem=43.93 GB): 100%|██████████| 4/4 [00:00<00:00, 53.12it/s]


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).